In [0]:
import pandas as pd

# Excel file-a pandas la read panrathu
file_path = "/Volumes/workspace/default/practice-files/sql sample.xlsx"
pandas_df = pd.read_excel(file_path)

# Pandas DataFrame-a Spark DataFrame-ku convert panrathu
df = spark.createDataFrame(pandas_df)

# Display panrathu
# display(df)

# Schema-va paakalam
df.printSchema()

# Row count check panalam
print(f"Total rows: {df.count()}")
display(df.tail(20))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType

spark = SparkSession.builder.appName(
    'Read CSV File into DataFrame') \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.caseSensitive", "True") \
    .getOrCreate()

df = spark.read.json("s3://siva-databricks-files/cyber-01/*")

df1 = df.withColumn(
    "as_date",
    to_timestamp(col("ITEM_DATE"), "dd-MMM-yy hh:mm:ss a")
)

def getInterval(time):
    start = int(time.split(":")[0])
    return f"{start}-{start+1}"

getIntervalUdf = udf(getInterval, StringType())

df2 = df1.withColumn("date", date_format(col('as_date'), 'yyyy-MM-dd')) \
         .withColumn("time", date_format(col('as_date'), 'HH:mm:ss a')) \
         .withColumn("Interval", getIntervalUdf("time")) \
         .withColumn("account_id", col('ACCOUNT_ID').cast(IntegerType()))

item_df1 = df2.groupBy('date','account_id','Interval').agg(count('*').alias('msg_per_hr'))         

# display(item_df1)

df3 = item_df1.withColumn("year", year(col('date').cast("timestamp")))\
    .withColumn('month', month(col('date').cast('timestamp')))\
    .withColumn("day", dayofmonth(col('date').cast('timestamp')))\
    .withColumn("hour", split(col('Interval'),'-')[0].cast(IntegerType()))\
    .withColumn("hour_interval", col('Interval'))\
    .withColumn("dayname", date_format(df2.date, "EEEE"))\
    .withColumn("week",weekofyear(col('date')))
display(df3)    


# df4=df3.withColumn("daytime",when((df3.hour >6) & (df3.hour <= 12), 'Morning').when((df3.hour > 12) & (df3.hour <= 17), 'Afternoon').when((df3.hour > 17) & (df3.hour <= 20), 'Evening').otherwise('Night'))
# display(df4)

# s3_bucket = 'siva-databricks-files'
# path = "s3://siva-databricks-files/cyber-01/"


# # df4.write.option("compression", "snappy").option("overwriteSchema", "true").option("path", f"3://{s3_bucket}/cyber-01/sgc_items/").mode("overwrite").format("parquet")
# df4.write.parquet(f"s3://{s3_bucket}/cyber-01/sgc_items/")
# print("done")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.window import Window

spark = SparkSession.builder.appName(
    'Read CSV File into DataFrame'
).config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
.config("spark.sql.caseSensitive", "True") \
.config("spark.sql.source.commitProtocolClass",
        "org.apache.spark.internal.io.cloud.PathOutputCommitProtocol") \
.getOrCreate()

ipath = "s3://siva-databricks-files/cyber-01/sgc_items/part-00000-tid-4466780840797933117-09085b49-498d-48c8-a2de-b8a028ef6d9d-174-1.c000.snappy.parquet"
df = spark.read.parquet(ipath)
# day = df.groupBy('date','Interval', 'hour', 'account_id', 'msg_per_hr').count()
display(df)

In [0]:
a='a3b2c4'
output = ".join(map(lamda x:x[0]*int (x[1], zip (s:[::2], s[1::2]))))"

s = 'a3b2c4'
output = ''.join(map(lambda x: x[0] * int(x[1]), zip(s[::2], s[1::2])))
print(output)
